In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

# Get Data

In [4]:
path = '/Users/danielriojas/Downloads/twitter_reddit_clean.csv'
data = pd.read_csv(f'{path}')
data.drop(columns=['Unnamed: 0'], inplace=True)
data.dropna(inplace=True)
data.head()

,text,label
0,god morning stil excited schol people,0
1,never felt low life wish easy way kil like fal...,1
2,haha love hate relationship yelow great doesnt...,0
3,day legendary season finale quot met mother quot,0
4,completely oposite girl say swet stuf thought ...,1


In [5]:
X = data.text
y = data.label

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X , y ,test_size = 0.3 )

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((17698,), (7586,), (17698,), (7586,))

In [8]:
X_train.head()

9401     ugh two smal plane row fun hour half il beach ...
1889     fel like runing time carer family wish solutio...
21883    im gon na touch zack brandon inapropriatly lat...
18323                            time go home later wekend
13110      whats name nice u met yal talk about wat ah tel
Name: text, dtype: object

# Tokenize and Word2Vec

In [11]:
X_train = [word_tokenize(str(_)) for _ in X_train]
X_test = [word_tokenize(str(_)) for _ in X_test]

In [12]:
#Embedding

word2vec = Word2Vec(sentences=X_train, vector_size=50, window=5, min_count=5)

In [22]:
word2vec.wv['suicide']

array([-0.24887732,  0.6983474 ,  0.671202  ,  0.7075017 ,  0.35201764,
       -0.55881214,  0.03826573, -0.15018763, -1.5827969 , -1.0676862 ,
       -0.39285755, -1.6639117 , -0.0412511 ,  0.45399556,  0.83822614,
       -0.17862618, -0.58707106, -0.928176  ,  0.2478188 ,  0.4874533 ,
       -0.87781435,  0.10514634,  0.29012215, -0.21728112, -0.5296157 ,
       -0.10905806, -0.95561606,  0.8808429 ,  0.30353117,  1.9301002 ,
        0.13734794, -0.27614504,  0.09580864, -0.22290432, -2.1173937 ,
        0.27816367,  0.742856  ,  0.1366753 ,  0.24063776, -1.7782966 ,
        1.173781  , -0.11440533,  0.08607572, -0.66691196,  0.3845871 ,
       -0.45914698, -0.23632964, -1.5191913 , -0.23470284,  0.29659158],
      dtype=float32)

In [23]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

# Padding

In [24]:
#dejamos en 400 maxlen

X_train_pad = pad_sequences(X_train, dtype='float', padding='post', maxlen= 400)
X_test_pad = pad_sequences(X_test, dtype='float', padding='post', maxlen = 400)

# RNN

In [30]:
def init_model():
    model = Sequential()
    model.add(Masking())
    model.add(LSTM(20, activation='tanh'))
    model.add(Dense(10, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics='accuracy')
    
    return model

model = init_model()

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

history_fit = model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es],
          verbose = 1
         )

Epoch 1/100
388/388 [==============================] - 88s 211ms/step - loss: 0.1508 - accuracy: 0.9533 - val_loss: 0.0834 - val_accuracy: 0.9733
Epoch 2/100
388/388 [==============================] - 73s 189ms/step - loss: 0.0740 - accuracy: 0.9756 - val_loss: 0.0913 - val_accuracy: 0.9714
Epoch 3/100
388/388 [==============================] - 72s 185ms/step - loss: 0.0676 - accuracy: 0.9781 - val_loss: 0.0704 - val_accuracy: 0.9765
Epoch 4/100
388/388 [==============================] - 76s 197ms/step - loss: 0.0620 - accuracy: 0.9810 - val_loss: 0.1045 - val_accuracy: 0.9638
Epoch 5/100
388/388 [==============================] - 75s 193ms/step - loss: 0.0599 - accuracy: 0.9801 - val_loss: 0.0701 - val_accuracy: 0.9776
Epoch 6/100
388/388 [==============================] - 76s 196ms/step - loss: 0.0566 - accuracy: 0.9817 - val_loss: 0.0793 - val_accuracy: 0.9740
Epoch 7/100
388/388 [==============================] - 79s 204ms/step - loss: 0.0547 - accuracy: 0.9824 - val_loss: 0.0725 -

TypeError: 'History' object is not subscriptable